In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr
import importlib
from obp.ope import (
    ContinuousOffPolicyEvaluation,
    KernelizedInverseProbabilityWeighting,\
)
import wandb

%matplotlib widget

ImportError: cannot import name 'ContinuousOffPolicyEvaluation' from 'obp.ope' (/Users/aakashsasikumar/.pyenv/versions/3.10.0/envs/ep_policy_diversity/lib/python3.10/site-packages/obp/ope/__init__.py)

In [ ]:
policy_list = sorted(list(glob.glob(f"../policy_library_20220820/**.pth")))

In [3]:
log_data = pd.read_csv("../data/rule_based_log_data/doee/0_cleaned_log.csv")
with open("../data/rule_based_log_data/doee/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
# with open("../data/invalid_policy_list_20220705.json") as f:
#     invalid_policies = json.load(f)["invalid_policies"]
invalid_policies = []

In [4]:
num_ts_per_day = 4 * 24
num_days = 15
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

In [5]:
policy_list = sorted(list(glob.glob(f"../policy_library_20220820/**.pth")))

In [6]:
def get_policy_scores(config, use_progress_bar=False):
    policy_scores = {}
    kernel = config["kernel"]
    bandwidth = config["bandwidth"]
    for zone in zones:
        # print(zone)
        ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
        ope_data = log_data[:ts_end]
        states = []
        actions = []
        rewards = []
        for i, row in ope_data.iterrows():
            state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                          "zone_humidity", "zone_temp", "zone_occupancy"]
            state = [row[var] for var in state_vars]
            action = row["action"]
            reward = row["reward"]
            states.append(state)
            rewards.append(reward)
            actions.append(action)
        ope = ContinuousOffPolicyEvaluation(bandit_feedback=
                                            {"action": np.array(actions),
                                             "reward": np.array(rewards),
                                             "pscore": np.ones((len(ope_data)))},
                                            ope_estimators=[KernelizedInverseProbabilityWeighting(kernel=kernel, bandwidth=bandwidth)])
        
        if use_progress_bar:
            policy_iterable = tqdm(policy_list)
        else:
            policy_iterable = policy_list
        for policy in policy_iterable:
            agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                        has_continuous_action_space=True, action_std_init=0.2, 
                        device=torch.device('cpu'), diverse_policies=list(),
                        diverse_weight=0, diverse_increase=True)
            agent.load(policy)
            agent.policy_evaluation = False
            agent.policy_old.set_action_std(0.1)

            # probs = torch.exp(agent.buffer.logprobs[0].reshape(-1, 1))
            if policy not in invalid_policies:
                # score, _, _, _, _ = ipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
                eval_actions = torch.Tensor(agent.select_action(states)).sigmoid()
                estimated_value = ope.estimate_policy_values(action_by_evaluation_policy=eval_actions.numpy())
                # print(estimated_value)
            else:
                continue
            if policy not in policy_scores:
                policy_scores[policy] = {}
            if zone not in policy_scores[policy]:
                policy_scores[policy][zone] = estimated_value["kernelized_ipw"]
    return policy_scores

In [7]:
def calculate_zonewise_spearman_corr(policy_scores):
    eval_data_loc = "../data/evaluation_report_20220820.csv"
    df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
    spearman_corr = {}
    for zone in zones:
        eval_df = df[df["zone"] == zone]

        # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
        for i_policy in invalid_policies:
            eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
        eval_df = eval_df.sort_values(by=["energy"])
        score_list = []
        for i, row in eval_df.iterrows():
            score_list.append(policy_scores[f"../{row['policy']}"][zone])

        eval_df["ope_scores"] = score_list
        correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values, nan_policy="omit")
        spearman_corr[zone] = correlation
    return spearman_corr

In [8]:
def calculate_average_spearman_corr(config):
    policy_scores = get_policy_scores(config)
    spearman_corr = calculate_zonewise_spearman_corr(policy_scores)
    corrs = []
    for zone in spearman_corr:
        corrs.append(abs(spearman_corr[zone].correlation))
    return np.mean(corrs)

In [9]:
config = {
    "kernel": "gaussian",
    "bandwidth": 0.3
}
policy_scores = get_policy_scores(config, use_progress_bar=True)

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

...

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

  0%|          | 0/870 [00:00<?, ?it/s]

In [10]:
spearman_corr = calculate_zonewise_spearman_corr(policy_scores)
spearman_corr

FileNotFoundError: [Errno 2] No such file or directory: '../data/evaluation_report_20220820.csv'

In [11]:
with open(f"data/doee/ContinuousActionGaussian/raw_scores/GaussianKernel_0.03_{num_days}_days_14_09_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores, f)

# with open(f"data/doee/ContinuousActionGaussian/spearman_corr/GaussianKernel_0.03_{num_days}_days_14_09_2022.pkl", "wb+") as f:
#     pickle.dump(spearman_corr, f)

## Wandb Hyperparameter Sweep

In [17]:
def evaluate_hyperparams(config=None):
    with wandb.init(config=config):
        config = wandb.config
        avg_spearman_corr = calculate_average_spearman_corr(config)
        wandb.log({"average_spearman_correlation": avg_spearman_corr})   

In [18]:
project = "ContinuousOPEHyperParamTuning_28_07_2022"

wandb.init(project=project)

sweep_config = {
    "method": "random",
    "metric": {
        "name": "average_spearman_correlation",
        "goal": "maximize"
    },
    "parameters": {
        "kernel": {
            "values": ["gaussian", "epanechnikov", "triangular", "cosine"]
        },
        "bandwidth": {
            "distribution": "uniform",
            "min": 0,
            "max": 0.5
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project=project)

wandb: Currently logged in as: aakashsasikumar. Use `wandb login --relogin` to force relogin


Create sweep with ID: qcxa15qo
Sweep URL: https://wandb.ai/aakashsasikumar/ContinuousOPEHyperParamTuning_28_07_2022/sweeps/qcxa15qo


In [19]:
wandb.agent(sweep_id, evaluate_hyperparams, count=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: cz9bnc8i with config:
wandb: 	bandwidth: 0.15177222212586255
wandb: 	kernel: cosine


average_spearman_correlation,▁
average_spearman_correlation,0.70286


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: w1viky0n with config:
wandb: 	bandwidth: 0.10137394054261668
wandb: 	kernel: epanechnikov


average_spearman_correlation,▁
average_spearman_correlation,0.60807


wandb: Agent Starting Run: zs7l3d5n with config:
wandb: 	bandwidth: 0.45405500840241136
wandb: 	kernel: triangular


...

wandb: Agent Starting Run: 5zybs0eu with config:
wandb: 	bandwidth: 0.25283714598479745
wandb: 	kernel: gaussian


average_spearman_correlation,▁
average_spearman_correlation,0.91647


wandb: Agent Starting Run: azs6q4ev with config:
wandb: 	bandwidth: 0.06546242995705204
wandb: 	kernel: cosine


average_spearman_correlation,▁
average_spearman_correlation,0.47896


wandb: Network error (ReadTimeout), entering retry loop.
